## POS tagging using  vanila viterbi & modified Viterbi

#### Note: Please run the whole note book 

In [1]:
#Importing libraries
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import seaborn
from collections import Counter

pd.set_option('display.max_columns',5400)
pd.set_option('display.max_rows',5400)

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
nltk_data[:10]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

#### All words are tagged. 
#### words are theirs taggs are inside a tuple
#### A sentence is represented as a list which congtains tuple of all words with taggs
#### The whole Treebank is inside a list
#### '.' can be considered as start. i.e. P(tag|start) = P(tag|.)

In [3]:
len(nltk_data)

3914

In [4]:
#spliting into train_test

random.seed(100)
train_data,test_data=train_test_split(nltk_data,test_size=0.05,random_state=50)
print('Length of train data = ',len(train_data))
print('Length of test data = ',len(test_data))

Length of train data =  3718
Length of test data =  196


In [5]:
#creating a list of tagged word

train_tagd=[tw for sent in train_data for tw in sent]
test_tagd=[tw for sent in test_data for tw in sent]
print(len(train_tagd))
print(len(test_tagd))

95421
5255


In [6]:
train_tagd[0:15] #(word,POS tag)

[('The', 'DET'),
 ('problem', 'NOUN'),
 ('involves', 'VERB'),
 ('the', 'DET'),
 ('motion', 'NOUN'),
 ('of', 'ADP'),
 ('small', 'ADJ'),
 ('magnetic', 'ADJ'),
 ('fields', 'NOUN'),
 ('within', 'ADP'),
 ('superconductor', 'NOUN'),
 ('crystals', 'NOUN'),
 (',', '.'),
 ('*', 'X'),
 ('limiting', 'VERB')]

In [7]:
# creating a vocabulary by extracting unique tokens(words,symbols,numbers) and unique POS tags

#tokens
V=set([i[0] for i in train_tagd])

#POS tags
T = set([i[1] for i in train_tagd])

print('number of unique token = ',len(V))
print('number of unique POS tags = ', len(T))


number of unique token =  12059
number of unique POS tags =  12


In [8]:
# unique pos tags are
T

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [9]:
# Creating parameters for HMM model

# Emission Probability

def w_g_t ( word,tag, train_d=train_tagd):
    onlyt=[wt for wt in train_d if wt[1]==tag ]
    w=[w for w in onlyt if w[0]==word]
    emission = len(w)/len(onlyt)
    
    return emission

#Transition probability

def t2_g_t1(t2,t1,train_d=train_tagd):
    tags=[i[1] for i in train_d]
    t1_c=[t for t in tags if t == t1]
    t2_t1_c=0
    for i in range(len(tags)-1):
        if tags[i]==t1 and tags[i+1] ==t2 :
            t2_t1_c+=1
            
    transition=t2_t1_c/len(t1_c)
            
    return (transition)


        
    
    

In [10]:
# creating a matrix to represent the relationship P(t2|t1) TxT MATRIX

tags_matrix=np.zeros((len(T),len(T)),dtype='float32')
tags_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [11]:
for i,t1 in enumerate(list(T)):
    for j,t2 in enumerate(list(T)):
        tags_matrix[i,j]=t2_g_t1(t2,t1)
tags_matrix

array([[8.04906059e-03, 4.98275179e-03, 4.86776531e-01, 2.08892301e-01,
        4.10118811e-02, 6.89919526e-03, 2.29973178e-02, 9.19892713e-02,
        1.22652361e-02, 3.37293968e-02, 9.58221499e-03, 7.28248358e-02],
       [6.17110804e-02, 4.67508187e-04, 1.54745206e-01, 3.51566166e-01,
        3.45956050e-02, 4.16082293e-02, 5.32959327e-02, 7.94763863e-03,
        4.67508193e-03, 5.28284237e-02, 1.18747078e-01, 1.17812060e-01],
       [3.56558971e-02, 5.29388851e-03, 1.69015184e-01, 1.10782407e-01,
        3.47995311e-02, 2.30439864e-02, 9.15531367e-02, 2.17983648e-01,
        3.11405212e-02, 8.10432062e-02, 1.34293497e-01, 6.53950945e-02],
       [4.64249169e-03, 4.24769707e-02, 1.46329880e-01, 2.64037132e-01,
        2.41373003e-01, 9.46775824e-03, 1.76853344e-01, 2.88784914e-02,
        4.35370654e-02, 1.73270945e-02, 1.30501539e-02, 1.20266117e-02],
       [6.54591098e-02, 5.76578192e-02, 8.85939747e-02, 2.22919658e-01,
        9.32568163e-02, 8.08823556e-02, 9.08357278e-02, 2.72

In [12]:
# creating a DataFrame
tags_df= pd.DataFrame(tags_matrix,columns=list(T),index=list(T))
tags_df

,PRON,CONJ,VERB,NOUN,.,NUM,ADP,X,PRT,ADV,DET,ADJ
PRON,0.008049,0.004983,0.486777,0.208892,0.041012,0.006899,0.022997,0.091989,0.012265,0.033729,0.009582,0.072825
CONJ,0.061711,0.000468,0.154745,0.351566,0.034596,0.041608,0.053296,0.007948,0.004675,0.052828,0.118747,0.117812
VERB,0.035656,0.005294,0.169015,0.110782,0.034800,0.023044,0.091553,0.217984,0.031141,0.081043,0.134293,0.065395
NOUN,0.004642,0.042477,0.146330,0.264037,0.241373,0.009468,0.176853,0.028878,0.043537,0.017327,0.013050,0.012027
.,0.065459,0.057658,0.088594,0.222920,0.093257,0.080882,0.090836,0.027260,0.002421,0.052188,0.173332,0.045104
NUM,0.001489,0.013992,0.017267,0.351295,0.118190,0.185770,0.034832,0.211075,0.027985,0.002679,0.002977,0.032450
ADP,0.070205,0.000856,0.008348,0.321811,0.039491,0.061537,0.016802,0.034996,0.001391,0.014127,0.325557,0.104880
X,0.055680,0.010689,0.203893,0.062699,0.164327,0.002872,0.144065,0.075144,0.184429,0.025526,0.054882,0.015795
PRT,0.017451,0.001646,0.403029,0.246954,0.043464,0.058281,0.020086,0.014159,0.001976,0.009549,0.100428,0.082977
ADV,0.014657,0.006995,0.345769,0.031646,0.134910,0.031646,0.119587,0.022985,0.014657,0.079614,0.068621,0.128914


In [13]:
# CREATING THE HMM-VITERBI ALGORITHM ( VANILA VITERBI FORM)

def h_viterbi(words,train_bag=train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    
    for index, word in enumerate(words):
        state_p=[]
        for tag in T:
            if index ==0:
                transition_p=tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[pos[-1],tag]
                
            #emission probability
            emission_p = w_g_t(words[index],tag)
            state=transition_p * emission_p
            state_p.append(state)
            
        max_p=max(state_p) #evaluating maximum likelyhood
        
        w_pos=T[state_p.index(max_p)]
        pos.append(w_pos)
    
    return list(zip(words,pos))

In [14]:
#creating tokens 

test_words=[i[0] for i in test_tagd]
test_words[:10]

['Editorials',
 'in',
 'the',
 'Greenville',
 'newspaper',
 'allowed',
 'that',
 'Mrs.',
 'Yeargin',
 'was']

In [15]:
test_out=h_viterbi(test_words)
test_out[:25]

[('Editorials', 'PRON'),
 ('in', 'ADP'),
 ('the', 'DET'),
 ('Greenville', 'NOUN'),
 ('newspaper', 'NOUN'),
 ('allowed', 'VERB'),
 ('that', 'ADP'),
 ('Mrs.', 'NOUN'),
 ('Yeargin', 'NOUN'),
 ('was', 'VERB'),
 ('wrong', 'ADJ'),
 (',', '.'),
 ('but', 'CONJ'),
 ('also', 'ADV'),
 ('said', 'VERB'),
 ('0', 'X'),
 ('the', 'DET'),
 ('case', 'NOUN'),
 ('showed', 'VERB'),
 ('how', 'ADV'),
 ('testing', 'NOUN'),
 ('was', 'VERB'),
 ('being', 'VERB'),
 ('overused', 'PRON'),
 ('*-2', 'X')]

In [16]:
#checking accuracy using base HMM & VITERBI algorithm

chk_tags = [i for i, j in zip(test_out, test_tagd) if i == j] 

accuracy = len(chk_tags)/len(test_out)
accuracy

0.9035204567078973

## Laplace Smoothing to deal with unknown words
### So when an unknown word appear in the test data 
#### there are chances that the transition probability may be zero
#### the emission probability will definitely be zero
#### i.e. P(T2|T1) = 0 , if we did not have the sequence T1 -> T2 in the training data. As all P(T2|T1) are non zeros, we do not need to modify the transition probability.
#### P(W|T) = 0 , as we do not have the word in out training data
#### here T - tag of unknow word W 

#### Emission Probability

#### P(W|T) = ((# W as T)+K ) / (#T + K* #words )

#### WHERE,
#### #words - total number of words vailable in the vocabulary
#### K - a value bw=etween 0-1 acts as a weight given to unknown values
#### (# W as T) - nuber of time W is tagged as T

In [17]:
# modified Emission Probability

def w_g_t_md( word,tag,k=1,train_d=train_tagd):
    T=list(set([i[1] for i in train_d]))
    wtr=list(set([i[0] for i in train_d]))
    onlyt=[wt for wt in train_d if wt[1]==tag ]
    w=[w for w in onlyt if w[0]==word]
    
    emission = (len(w)+k)/(len(onlyt)+k*len(wtr))
    return emission
    
    

In [18]:
# MODIFIED HMM-VITERBI ALGORITHM ( BASE FORM) according to Laplace smoothing

def h_viterbi_lp(words,train_bag =train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    
    for index, word in enumerate(words):
        state_p=[]
        for tag in T:
            if index ==0:
                transition_p=tags_df.loc['.',tag]
            else:
                transition_p = tags_df.loc[pos[-1],tag]
                
            #emission probability
            emission_p = w_g_t_md(words[index],tag)
            state=transition_p * emission_p
            state_p.append(state)
            
        max_p=max(state_p) #evaluating maximum likelyhood
        
        w_pos=T[state_p.index(max_p)]
        pos.append(w_pos)
    
    return list(zip(words,pos))

In [19]:
#checking accuracy
test_lp = h_viterbi_lp(test_words)
chk_tags = [i for i, j in zip(test_lp, test_tagd) if i == j] 

accuracy_lp = len(chk_tags)/len(test_lp)
accuracy_lp

0.8896289248334919

In [20]:
# incorrect tag cases for base 
incorrect_tagged_cases = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_out, test_tagd)) if j[0]!=j[1]]

In [21]:
# incorrect tag cases for lp
incorrect_tagged_cases_lp = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_lp, test_tagd)) if j[0]!=j[1]]

In [22]:
incorrect_tagged_cases

[[("''", '.'),
  (('Editorials', 'PRON'), ('Editorials', 'NOUN')),
  ('in', 'ADP')],
 [('being', 'VERB'),
  (('overused', 'PRON'), ('overused', 'VERB')),
  ('*-2', 'X')],
 [('risen', 'VERB'), (('smartly', 'PRON'), ('smartly', 'ADV')), (',', '.')],
 [('with', 'ADP'), (('even', 'ADV'), ('even', 'ADJ')), ('stronger', 'ADJ')],
 [('will', 'VERB'), (('likely', 'ADJ'), ('likely', 'ADV')), ('cause', 'VERB')],
 [('likely', 'ADV'), (('cause', 'NOUN'), ('cause', 'VERB')), ('the', 'DET')],
 [('No', 'DET'), (('one', 'NUM'), ('one', 'NOUN')), ('is', 'VERB')],
 [('employs', 'VERB'),
  (('2,700', 'PRON'), ('2,700', 'NUM')),
  ('people', 'NOUN')],
 [('$', '.'), (('370', 'PRON'), ('370', 'NUM')), ('million', 'NUM')],
 [('*', 'X'),
  (('reinstating', 'PRON'), ('reinstating', 'VERB')),
  ('a', 'DET')],
 [('public', 'ADJ'),
  (('cold', 'NOUN'), ('cold', 'ADJ')),
  ('shoulder', 'NOUN')],
 [('$', '.'), (('14.75', 'PRON'), ('14.75', 'NUM')), ('*U*', 'X')],
 [('is', 'VERB'), (('furious', 'PRON'), ('furious', '

In [23]:
incorrect_tagged_cases_lp

[[("''", '.'), (('Editorials', 'DET'), ('Editorials', 'NOUN')), ('in', 'ADP')],
 [('how', 'ADV'), (('testing', 'VERB'), ('testing', 'NOUN')), ('was', 'VERB')],
 [('being', 'VERB'), (('overused', 'X'), ('overused', 'VERB')), ('*-2', 'X')],
 [('take', 'VERB'),
  (('extensive', 'X'), ('extensive', 'ADJ')),
  ('notes', 'NOUN')],
 [('risen', 'VERB'), (('smartly', 'X'), ('smartly', 'ADV')), (',', '.')],
 [('with', 'ADP'), (('even', 'ADV'), ('even', 'ADJ')), ('stronger', 'ADJ')],
 [('will', 'VERB'), (('likely', 'ADJ'), ('likely', 'ADV')), ('cause', 'VERB')],
 [('likely', 'ADV'), (('cause', 'NOUN'), ('cause', 'VERB')), ('the', 'DET')],
 [('No', 'DET'), (('one', 'NUM'), ('one', 'NOUN')), ('is', 'VERB')],
 [('which', 'DET'), (('*T*-26', 'NOUN'), ('*T*-26', 'X')), ('ought', 'VERB')],
 [('America', 'NOUN'), (('abroad', '.'), ('abroad', 'ADV')), (';', '.')],
 [('can', 'VERB'),
  (('American', 'NOUN'), ('American', 'ADJ')),
  ('taxpayers', 'NOUN')],
 [('questions', 'NOUN'), (('*T*-5', '.'), ('*T*-5'

In [24]:
#checking accuracy
print('accuracy of vanila viterbi form = ',accuracy)
print('accuracy of viterbi + laplace smoothing form = ',accuracy_lp)

accuracy of vanila viterbi form =  0.9035204567078973
accuracy of viterbi + laplace smoothing form =  0.8896289248334919


## Viterbi + ruled base tagging to deal with unknown words

In [25]:
# defining regexp for ruled base tagging

patterns = [
    (r'.*ing$', 'VERB'),              
    (r'.*ed$', 'VERB'),               
    (r'.*es$', 'VERB'),               
    (r'.*ould$', 'VERB'),              
    (r'.*\'s$', 'NOUN'),              
    (r'.*s$', 'NOUN'),                
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),
    (r'.*able$', 'ADJ'),
    (r'.*ly$', 'ADV'),
    (r'.*', 'NOUN')
      
]

regexp_tagger = nltk.RegexpTagger(patterns)

In [26]:
# regexp_tagger.tag(['Editorials'])[0][1]

In [27]:
# Viterbi + Rule based tagger 

def h_viterbi_rb(words,train_bag=train_tagd):
    pos=[]
    T=list(set([i[1] for i in train_bag]))
    wtr=list(set([i[0] for i in train_bag]))
    
    for index, word in enumerate(words):

        if word in wtr:
            state_p=[]
            for tag in T:
                if index ==0:
                    transition_p=tags_df.loc['.',tag]
                else:
                    transition_p = tags_df.loc[pos[-1],tag]

                #emission probability
                emission_p = w_g_t(words[index],tag)
                state=transition_p * emission_p
                state_p.append(state)

            max_p=max(state_p) #evaluating maximum likelyhood

            w_pos=T[state_p.index(max_p)]
            pos.append(w_pos)
        else:
            wordx=[]
            wordx.append(word)
            w_pos=regexp_tagger.tag(wordx)
            pos.append(w_pos[0][1])
            
    
    return list(zip(words,pos))

In [28]:
#checking accuracy

test_rb = h_viterbi_rb(test_words)
chk_tags = [i for i, j in zip(test_rb, test_tagd) if i == j] 

accuracy_rb = len(chk_tags)/len(test_rb)
accuracy_rb

0.9493815413891532

In [29]:
# incorrect tag cases for rb
incorrect_tagged_cases_rb = [[test_tagd[i-1],j,test_tagd[i+1]] for i, j in enumerate(zip(test_rb, test_tagd)) if j[0]!=j[1]]

### Accuracy comparison

In [30]:
# accuracy comparison

print('accuracy of vanila viterbi form = ',accuracy)
print('accuracy of viterbi + laplace smoothing form = ',accuracy_lp)
print('accuracy of viterbi + rule based form = ',accuracy_rb)

#Viterbi + rule based form has high tagging accuracy

accuracy of vanila viterbi form =  0.9035204567078973
accuracy of viterbi + laplace smoothing form =  0.8896289248334919
accuracy of viterbi + rule based form =  0.9493815413891532


## Checking with test samples:

In [32]:
# checking with test samples

text=open('Test_sentences.txt').read()
text

"Android is a mobile operating system developed by Google.\nAndroid has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\nGoogle and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\nTwitter is an online news and social networking service on which users post and interact with messages known as tweets.\nBefore entering politics, Donald Trump was a domineering businessman and a television personality.\nThe 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\nThis is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\nShow me the cheapest round trips from Dallas to Atlanta\nI would like to see flights from Denver to Philadelphia.\nShow me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\nNASA invited social media users to experience the launch of ICESAT-2 Satell

In [33]:
# creating tokens
token_txt=word_tokenize(text)
token_txt

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since',
 '2011',
 'and',
 'on',
 'tablets',
 'since',
 '2013',
 '.',
 'Google',
 'and',
 'Twitter',
 'made',
 'a',
 'deal',
 'in',
 '2015',
 'that',
 'gave',
 'Google',
 'access',
 'to',
 'Twitter',
 "'s",
 'firehose',
 '.',
 'Twitter',
 'is',
 'an',
 'online',
 'news',
 'and',
 'social',
 'networking',
 'service',
 'on',
 'which',
 'users',
 'post',
 'and',
 'interact',
 'with',
 'messages',
 'known',
 'as',
 'tweets',
 '.',
 'Before',
 'entering',
 'politics',
 ',',
 'Donald',
 'Trump',
 'was',
 'a',
 'domineering',
 'businessman',
 'and',
 'a',
 'television',
 'personality',
 '.',
 'The',
 '2018',
 'FIFA',
 'World',
 'Cup',
 'is',
 'the',
 '21st',
 'FIFA',
 'World',
 'Cup',
 ',',
 'an',
 'international',
 'football',
 'tournament',
 'contested',
 'once',
 'every',
 'four',
 'years',
 '.'

In [34]:
# tagging by vanila viterbi algorithm

st=h_viterbi(token_txt)
st

[('Android', 'PRON'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'PRON'),
 ('.', '.'),
 ('Android', 'PRON'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'PRON'),
 ('OS', 'PRON'),
 ('worldwide', 'PRON'),
 ('on', 'ADP'),
 ('smartphones', 'PRON'),
 ('since', 'ADP'),
 ('2011', 'PRON'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'PRON'),
 ('.', '.'),
 ('Google', 'PRON'),
 ('and', 'CONJ'),
 ('Twitter', 'PRON'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'PRON'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'PRON'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'PRON'),
 ("'s", 'VERB'),
 ('firehose', 'PRON'),
 ('.', '.'),
 ('Twitter', 'PRON'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'PRON'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),

In [35]:
# tgging by modified viterbi algorithm (laplace smoothing)
lp=h_viterbi_lp(token_txt)
lp

[('Android', 'DET'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'DET'),
 ('.', '.'),
 ('Android', 'DET'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'NOUN'),
 ('OS', '.'),
 ('worldwide', 'DET'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'NOUN'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'DET'),
 ('.', '.'),
 ('Google', 'DET'),
 ('and', 'NOUN'),
 ('Twitter', '.'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'DET'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'DET'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOU

In [36]:
#tagging by viterbi + rule base tagger

rb=h_viterbi_rb(token_txt)
rb

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'VERB'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('

In [37]:
#comparison

#EXTRACTING POS TAGS
st_tags=[i[1] for i in st]
lp_tags=[i[1] for i in lp]
rb_tags=[i[1] for i in rb]


# Many words(such as Android, Google,NASA,Satellite, etc.) which were tagged wrongly by vanila viterbi are corrected by viterbi + rule based tagger

compare=pd.DataFrame({'Words':token_txt,'vanila viterbi':st_tags,'Laplace viterbi':lp_tags,'rule based viterbi':rb_tags})
compare

,Words,vanila viterbi,Laplace viterbi,rule based viterbi
0,Android,PRON,DET,NOUN
1,is,VERB,VERB,VERB
2,a,DET,DET,DET
3,mobile,ADJ,ADJ,ADJ
4,operating,NOUN,NOUN,NOUN
5,system,NOUN,NOUN,NOUN
6,developed,VERB,VERB,VERB
7,by,ADP,ADP,ADP
8,Google,PRON,DET,NOUN
9,.,.,.,.
